# AutoGen Agents στην Παραγωγή: Παρατηρησιμότητα & Αξιολόγηση

Σε αυτό το σεμινάριο, θα μάθουμε πώς να **παρακολουθούμε τα εσωτερικά βήματα (ίχνη) των [Autogen agents](https://github.com/microsoft/autogen)** και πώς να **αξιολογούμε την απόδοσή τους** χρησιμοποιώντας το [Langfuse](https://langfuse.com).

Αυτός ο οδηγός καλύπτει **online** και **offline** μετρικές αξιολόγησης που χρησιμοποιούνται από ομάδες για να φέρουν τους agents στην παραγωγή γρήγορα και αξιόπιστα.

**Γιατί η Αξιολόγηση AI Agents είναι σημαντική:**
- Εντοπισμός προβλημάτων όταν οι εργασίες αποτυγχάνουν ή παράγουν μη βέλτιστα αποτελέσματα
- Παρακολούθηση κόστους και απόδοσης σε πραγματικό χρόνο
- Βελτίωση αξιοπιστίας και ασφάλειας μέσω συνεχούς ανατροφοδότησης


## Βήμα 1: Ορισμός Μεταβλητών Περιβάλλοντος

Αποκτήστε τα API keys του Langfuse κάνοντας εγγραφή στο [Langfuse Cloud](https://cloud.langfuse.com/) ή [φιλοξενώντας το Langfuse μόνοι σας](https://langfuse.com/self-hosting).

_**Σημείωση:** Όσοι φιλοξενούν μόνοι τους μπορούν να χρησιμοποιήσουν [τα Terraform modules](https://langfuse.com/self-hosting/azure) για να εγκαταστήσουν το Langfuse στο Azure. Εναλλακτικά, μπορείτε να εγκαταστήσετε το Langfuse στο Kubernetes χρησιμοποιώντας το [Helm chart](https://langfuse.com/self-hosting/kubernetes-helm)._


In [5]:
import os

# Get keys for your project from the project settings page: https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..." 
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..." 
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com" # 🇪🇺 EU region
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

Με τις μεταβλητές περιβάλλοντος ρυθμισμένες, μπορούμε τώρα να αρχικοποιήσουμε τον πελάτη Langfuse. Το `get_client()` αρχικοποιεί τον πελάτη Langfuse χρησιμοποιώντας τα διαπιστευτήρια που παρέχονται στις μεταβλητές περιβάλλοντος.


In [6]:
from langfuse import Langfuse
 
# Filter out Autogen OpenTelemetryspans
langfuse = Langfuse(
    blocked_instrumentation_scopes=["autogen SingleThreadedAgentRuntime"]
)
 
# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

Langfuse client is authenticated and ready!


## Βήμα 2: Αρχικοποίηση της Εργαλειοθήκης OpenLit

Τώρα, αρχικοποιούμε την εργαλειοθήκη [OpenLit](https://github.com/openlit/openlit). Το OpenLit καταγράφει αυτόματα τις λειτουργίες AutoGen και εξάγει τα OpenTelemetry (OTel) spans στο Langfuse.


In [7]:
import openlit
 
# Initialize OpenLIT instrumentation. The disable_batch flag is set to true to process traces immediately.
openlit.init(tracer=langfuse._otel_tracer, disable_batch=True, disabled_instrumentors=["mistral"])

## Βήμα 3: Εκτελέστε τον πράκτορά σας

Τώρα ρυθμίζουμε έναν πράκτορα πολλαπλών βημάτων για να δοκιμάσουμε την οργάνωσή μας.


In [2]:
import os

from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_agentchat.base import TaskResult

from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

In [3]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
        "structured_output": False
    },
)

In [8]:
# 🍴 Agent 1 – proposes ONE healthy meal idea each turn
meal_planner_agent = AssistantAgent(
    "meal_planner_agent",
    model_client=client,
    description="A seasoned meal-planning coach who suggests balanced meals.",
    system_message="""
    You are a Meal-Planning Assistant with a decade of experience helping busy people prepare meals.
    Goal: propose the single best meal (breakfast, lunch, or dinner) given the user's context.
    Each response must contain ONLY one complete meal idea (title + very brief component list) — no extras.
    Keep it concise: skip greetings, chit-chat, and filler.
    """,
)

# 🥗 Agent 2 – checks nutritional quality & variety
nutritionist_agent = AssistantAgent(
    "nutritionist_agent",
    model_client=client,
    description="A registered dietitian ensuring meals meet nutritional standards.",
    system_message="""
    You are a Nutritionist focused on whole-food, macro-balanced eating.
    Evaluate the meal_planner_agent’s recommendation.
    If the meal is nutritionally sound, sufficiently varied, and portion-appropriate, respond with 'APPROVE'.
    Otherwise, give high-level guidance on how to improve it (e.g. 'add a plant-based protein') — do NOT provide a full alternative recipe.
    """,
)

In [9]:
# ✅ Chat stops once the nutritionist says APPROVE
termination = TextMentionTermination("APPROVE")

# 🔄 Alternate turns between the two agents until termination
team = RoundRobinGroupChat(
    [meal_planner_agent, nutritionist_agent],
    termination_condition=termination,
)

# Example kickoff
user_input = "I'm looking for a quick, delicious dinner I can prep after work. I have 30 minutes and minimal clean-up is ideal."

In [ ]:
with langfuse.start_as_current_span(name="create_meal_plan") as span:
    async for message in team.run_stream(task=user_input):
        if isinstance(message, TaskResult):
            print("Stop Reason:", message.stop_reason)
        else:
            print(message)

    span.update_trace(
        input=user_input,
        output=message.stop_reason,
    )

# Flush the trace to Langfuse for short-lived environments such as Jupyter Notebooks
langfuse.flush()

### Δομή Ιχνηλάτησης

Το Langfuse καταγράφει ένα **ίχνος** που περιέχει **διαστήματα**, τα οποία αντιπροσωπεύουν κάθε βήμα της λογικής του πράκτορά σας. Εδώ, το ίχνος περιλαμβάνει τη συνολική εκτέλεση του πράκτορα και υπο-διαστήματα για:
- Τον πράκτορα σχεδιασμού γευμάτων
- Τους πράκτορες διατροφολόγους

Μπορείτε να τα εξετάσετε για να δείτε ακριβώς πού ξοδεύεται χρόνος, πόσα tokens χρησιμοποιούνται, και ούτω καθεξής:

![Δέντρο ιχνηλάτησης στο Langfuse](https://langfuse.com/images/cookbook/example-autogen-evaluation/trace-tree.png)

_[Σύνδεσμος στο ίχνος](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64)_


## Αξιολόγηση Online

Η Αξιολόγηση Online αναφέρεται στην αξιολόγηση του πράκτορα σε ένα ζωντανό, πραγματικό περιβάλλον, δηλαδή κατά τη διάρκεια της πραγματικής χρήσης σε παραγωγή. Αυτό περιλαμβάνει την παρακολούθηση της απόδοσης του πράκτορα σε πραγματικές αλληλεπιδράσεις με χρήστες και τη συνεχή ανάλυση των αποτελεσμάτων.

### Συνηθισμένοι Δείκτες Παρακολούθησης στην Παραγωγή

1. **Κόστη** — Η καταγραφή δεδομένων περιλαμβάνει τη χρήση tokens, την οποία μπορείτε να μετατρέψετε σε κατά προσέγγιση κόστη, αναθέτοντας μια τιμή ανά token.
2. **Καθυστέρηση** — Παρακολουθήστε τον χρόνο που απαιτείται για την ολοκλήρωση κάθε βήματος ή ολόκληρης της διαδικασίας.
3. **Ανατροφοδότηση Χρηστών** — Οι χρήστες μπορούν να παρέχουν άμεση ανατροφοδότηση (θετική/αρνητική) για να βοηθήσουν στη βελτίωση ή τη διόρθωση του πράκτορα.
4. **LLM-ως-Κριτής** — Χρησιμοποιήστε ένα ξεχωριστό LLM για να αξιολογήσετε την έξοδο του πράκτορα σχεδόν σε πραγματικό χρόνο (π.χ., έλεγχος για τοξικότητα ή ορθότητα).

Παρακάτω, παρουσιάζουμε παραδείγματα αυτών των δεικτών.


#### 1. Κόστη

Παρακάτω υπάρχει ένα στιγμιότυπο οθόνης που δείχνει τη χρήση για κλήσεις `gpt-4o-mini`. Αυτό είναι χρήσιμο για να δείτε τα βήματα που κοστίζουν περισσότερο και να βελτιστοποιήσετε τον πράκτορά σας.

![Κόστη](https://langfuse.com/images/cookbook/example-autogen-evaluation/gpt-4o-costs.png) 

_[Σύνδεσμος στο ίχνος](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64)_


#### 2. Καθυστέρηση

Μπορούμε επίσης να δούμε πόσος χρόνος χρειάστηκε για να ολοκληρωθεί κάθε βήμα. Στο παρακάτω παράδειγμα, ολόκληρη η εκτέλεση πήρε περίπου 3 δευτερόλεπτα, τα οποία μπορείτε να αναλύσετε ανά βήμα. Αυτό σας βοηθά να εντοπίσετε σημεία συμφόρησης και να βελτιστοποιήσετε τον πράκτορά σας.

![Καθυστέρηση](https://langfuse.com/images/cookbook/example-autogen-evaluation/agent-latency.png) 

_[Σύνδεσμος στο ίχνος](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64?display=timeline)_


#### 3. Σχόλια Χρηστών

Εάν ο agent σας είναι ενσωματωμένος σε μια διεπαφή χρήστη, μπορείτε να καταγράψετε άμεσα σχόλια χρηστών (όπως ένα thumbs-up/down σε μια διεπαφή συνομιλίας).


In [10]:
from langfuse import get_client
 
langfuse = get_client()
 
# Option 1: Use the yielded span object from the context manager
with langfuse.start_as_current_span(
    name="autogen-request-user-feedback-1") as span:
    
    async for message in team.run_stream(task="Create a meal with potatoes"):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message)    
 
    # Score using the span object
    span.score_trace(
        name="user-feedback",
        value=1,
        data_type="NUMERIC",
        comment="This was delicious, thank you"
    )
 
# Option 2: Use langfuse.score_current_trace() if still in context
with langfuse.start_as_current_span(name="autogen-request-user-feedback-2") as span:
    # ... Autogen execution ...

    async for message in team.run_stream(task="I am allergic to gluten."):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message)    
 
    # Score using current context
    langfuse.score_current_trace(
        name="user-feedback",
        value=1,
        data_type="NUMERIC"
    )

id='da068880-22ae-4f01-9f01-2bb231939089' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 43, 732669, tzinfo=datetime.timezone.utc) content='Create a meal with potatoes' type='TextMessage'
id='ad937ce4-3534-493f-824b-ca9c226b5287' source='meal_planner_agent' models_usage=RequestUsage(prompt_tokens=95, completion_tokens=30) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 45, 186423, tzinfo=datetime.timezone.utc) content='Potato and Spinach Frittata  \n- Eggs  \n- Potatoes  \n- Fresh spinach  \n- Onion  \n- Cheese (optional)  ' type='TextMessage'
id='50fd33c1-057f-49fe-afad-ee86d164296d' source='nutritionist_agent' models_usage=RequestUsage(prompt_tokens=132, completion_tokens=4) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 45, 581059, tzinfo=datetime.timezone.utc) content='APPROVE' type='TextMessage'
Stop Reason: Text 'APPROVE' mentioned
id='e371de6c-e5fc-42c1-8eda-e5b8cd5accab' source='user' models_usage=None met

In [ ]:
# Option 3: Use create_score() with trace ID (when outside context)
langfuse.create_score(
    trace_id="predefined_trace_id",
    name="user-feedback",
    value=1,
    data_type="NUMERIC",
    comment="This was correct, thank you"
)

Η ανατροφοδότηση των χρηστών καταγράφεται στο Langfuse:

![Η ανατροφοδότηση των χρηστών καταγράφεται στο Langfuse](https://langfuse.com/images/cookbook/example-autogen-evaluation/user-feedback.png)


#### 4. Αυτοματοποιημένη Αξιολόγηση με LLM-as-a-Judge

Το LLM-as-a-Judge είναι ένας άλλος τρόπος για να αξιολογήσετε αυτόματα την έξοδο του πράκτορά σας. Μπορείτε να ρυθμίσετε μια ξεχωριστή κλήση LLM για να εκτιμήσετε την ορθότητα, την τοξικότητα, το ύφος ή οποιοδήποτε άλλο κριτήριο σας ενδιαφέρει.

**Ροή Εργασίας**:
1. Ορίζετε ένα **Πρότυπο Αξιολόγησης**, π.χ., "Ελέγξτε αν το κείμενο είναι τοξικό."
2. Ρυθμίζετε ένα μοντέλο που χρησιμοποιείται ως μοντέλο-κριτής· σε αυτή την περίπτωση το `gpt-4o-mini` μέσω Azure.
2. Κάθε φορά που ο πράκτοράς σας παράγει έξοδο, περνάτε αυτή την έξοδο στο "κριτή" LLM με το πρότυπο.
3. Το LLM-κριτής απαντά με μια βαθμολογία ή ετικέτα που καταγράφετε στο εργαλείο παρατηρησιμότητας.

Παράδειγμα από το Langfuse:

![LLM-as-a-Judge Evaluator](https://langfuse.com/images/cookbook/example-autogen-evaluation/evaluator.png)


In [12]:
with langfuse.start_as_current_span(name="autogen-request-user-feedback-2") as span:

    async for message in team.run_stream(task="I am a picky eater and not sure if you find something for me."):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message) 

    span.update_trace(
        input=user_input,
        output=message.stop_reason,
    )

langfuse.flush()

id='eefc628d-502f-451a-8f70-be486f62f8c5' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 29, 171393, tzinfo=datetime.timezone.utc) content='I am a picky eater and not sure if you find something for me.' type='TextMessage'
id='13b3e14b-bcf7-42a5-80d6-54b0c7be765e' source='meal_planner_agent' models_usage=RequestUsage(prompt_tokens=352, completion_tokens=27) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 30, 433516, tzinfo=datetime.timezone.utc) content='Chicken Alfredo Pasta  \n- Gluten-free pasta  \n- Grilled chicken breast  \n- Heavy cream  \n- Parmesan cheese  \n- Garlic  ' type='TextMessage'
id='550f2dee-0e08-4bbd-b67f-991b467328f1' source='nutritionist_agent' models_usage=RequestUsage(prompt_tokens=386, completion_tokens=17) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 31, 505173, tzinfo=datetime.timezone.utc) content='Consider incorporating some vegetables, like spinach or broccoli, to increase the nutrien

Μπορείτε να δείτε ότι η απάντηση αυτού του παραδείγματος κρίνεται ως "μη τοξική".

![Βαθμολογία Αξιολόγησης LLM-as-a-Judge](https://langfuse.com/images/cookbook/example-autogen-evaluation/llm-as-a-judge-score.png)


#### 5. Επισκόπηση Μετρικών Παρατηρησιμότητας

Όλες αυτές οι μετρικές μπορούν να απεικονιστούν μαζί σε πίνακες ελέγχου. Αυτό σας επιτρέπει να δείτε γρήγορα πώς αποδίδει ο agent σας σε πολλές συνεδρίες και σας βοηθά να παρακολουθείτε μετρικές ποιότητας με την πάροδο του χρόνου.

![Επισκόπηση μετρικών παρατηρησιμότητας](https://langfuse.com/images/cookbook/example-autogen-evaluation/dashboard.png)


## Αξιολόγηση Εκτός Σύνδεσης

Η αξιολόγηση σε πραγματικό χρόνο είναι απαραίτητη για άμεση ανατροφοδότηση, αλλά χρειάζεστε επίσης **αξιολόγηση εκτός σύνδεσης**—συστηματικούς ελέγχους πριν ή κατά τη διάρκεια της ανάπτυξης. Αυτό βοηθά στη διατήρηση της ποιότητας και της αξιοπιστίας πριν από την εφαρμογή αλλαγών στην παραγωγή.


### Αξιολόγηση Συνόλου Δεδομένων

Στην αξιολόγηση εκτός σύνδεσης, συνήθως:
1. Έχετε ένα σύνολο δεδομένων αναφοράς (με ζεύγη ερωτήσεων και αναμενόμενων απαντήσεων)
2. Εκτελείτε τον πράκτορά σας σε αυτό το σύνολο δεδομένων
3. Συγκρίνετε τα αποτελέσματα με τα αναμενόμενα ή χρησιμοποιείτε έναν επιπλέον μηχανισμό βαθμολόγησης

Παρακάτω, παρουσιάζουμε αυτήν την προσέγγιση με το [q&a-dataset](https://huggingface.co/datasets/junzhang1207/search-dataset), το οποίο περιέχει ερωτήσεις και αναμενόμενες απαντήσεις.


In [16]:
import pandas as pd
from datasets import load_dataset
 
# Fetch search-dataset from Hugging Face
dataset = load_dataset("junzhang1207/search-dataset", split = "train")
df = pd.DataFrame(dataset)
print("First few rows of search-dataset:")
print(df.head())

/Users/jannik/Documents/GitHub/ai-agents-for-beginners/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


First few rows of search-dataset:
                                     id  \
0  20caf138-0c81-4ef9-be60-fe919e0d68d4   
1  1f37d9fd-1bcc-4f79-b004-bc0e1e944033   
2  76173a7f-d645-4e3e-8e0d-cca139e00ebe   
3  5f5ef4ca-91fe-4610-a8a9-e15b12e3c803   
4  64dbed0d-d91b-4acd-9a9c-0a7aa83115ec   

                                            question  \
0                 steve jobs statue location budapst   
1  Why is the Battle of Stalingrad considered a t...   
2  In what year did 'The Birth of a Nation' surpa...   
3  How many Russian soldiers surrendered to AFU i...   
4   What event led to the creation of Google Images?   

                                     expected_answer       category       area  
0  The Steve Jobs statue is located in Budapest, ...           Arts  Knowledge  
1  The Battle of Stalingrad is considered a turni...   General News       News  
2  This question is based on a false premise. 'Th...  Entertainment       News  
3  About 300 Russian soldiers surrendered to t

Στη συνέχεια, δημιουργούμε μια οντότητα συνόλου δεδομένων στο Langfuse για να παρακολουθούμε τις εκτελέσεις. Έπειτα, προσθέτουμε κάθε στοιχείο από το σύνολο δεδομένων στο σύστημα.


In [17]:
from langfuse import Langfuse
langfuse = Langfuse()
 
langfuse_dataset_name = "qa-dataset_autogen-agent"
 
# Create a dataset in Langfuse
langfuse.create_dataset(
    name=langfuse_dataset_name,
    description="q&a dataset uploaded from Hugging Face",
    metadata={
        "date": "2025-03-21",
        "type": "benchmark"
    }
)

Dataset(id='cmcm7524d00kjad07s2cjwqcf', name='qa-dataset_autogen-agent', description='q&a dataset uploaded from Hugging Face', metadata={'date': '2025-03-21', 'type': 'benchmark'}, project_id='cloramnkj0002jz088vzn1ja4', created_at=datetime.datetime(2025, 7, 2, 16, 54, 7, 357000, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2025, 7, 2, 16, 54, 7, 357000, tzinfo=datetime.timezone.utc))

In [18]:
df_25 = df.sample(25) # For this example, we upload only 25 dataset questions

for idx, row in df_25.iterrows():
    langfuse.create_dataset_item(
        dataset_name=langfuse_dataset_name,
        input={"text": row["question"]},
        expected_output={"text": row["expected_answer"]}
    )

![Στοιχεία συνόλου δεδομένων στο Langfuse](https://langfuse.com/images/cookbook/example-autogen-evaluation/example-dataset.png)


#### Εκτέλεση του Agent στο Σύνολο Δεδομένων

Αρχικά, δημιουργούμε έναν απλό Autogen agent που απαντά σε ερωτήσεις χρησιμοποιώντας μοντέλα Azure OpenAI.


In [8]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage

In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    max_tokens=5000,
    model_info={
        "json_output": True,
        "function_calling": False,
        "vision": False,
        "family": "unknown",
        "structured_output": True,
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

In [18]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are participant in a quizz show and you are given a question. You need to create a short answer to the question.",
)

Στη συνέχεια, ορίζουμε μια βοηθητική συνάρτηση `my_agent()`.


In [19]:
async def my_agent(user_query: str):

    with langfuse.start_as_current_span(name="autogen-trace") as span:

        # Execute the agent response
        response = await agent.on_messages(
            [TextMessage(content=user_query, source="user")],
            cancellation_token=CancellationToken(),
        )

        span.update_trace(
            input=user_query,
            output=response.chat_message.content,
        )

    return str(response.chat_message.content)

# Test the function
await my_agent("What is the capital of France?")

'The capital of France is Paris.'

Τέλος, επαναλαμβάνουμε για κάθε στοιχείο του συνόλου δεδομένων, εκτελούμε τον πράκτορα και συνδέουμε το ίχνος με το στοιχείο του συνόλου δεδομένων. Μπορούμε επίσης να επισυνάψουμε μια γρήγορη βαθμολογία αξιολόγησης, αν το επιθυμούμε.


In [20]:
dataset_name = "qa-dataset_autogen-agent"
current_run_name = "dev_tasks_run-autogen_gpt-4.1" # Identifies this specific evaluation run
current_run_metadata={"model_provider": "Azure", "model": "gpt-4.1"}
current_run_description="Evaluation run for Autogen model on July 3rd"

dataset = langfuse.get_dataset('qa-dataset_autogen-agent')

for item in dataset.items:
    print(f"Running evaluation for item: {item.id} (Input: {item.input})")
 
    # Use the item.run() context manager
    with item.run(
        run_name=current_run_name,
        run_metadata=current_run_metadata,
        run_description=current_run_description
    ) as root_span: 
        # All subsequent langfuse operations within this block are part of this trace.
        generated_answer = await my_agent(user_query = item.input["text"])
    
    print("Generated Answer: ", generated_answer)
 
print(f"\nFinished processing dataset '{dataset_name}' for run '{current_run_name}'.")

langfuse.flush()

Running evaluation for item: 09810cc4-9992-4712-a3b2-7224da31776a (Input: {'text': 'In Hindu mythology, which deity is the Ganges river dolphin associated with?'})
Generated Answer:  In Hindu mythology, the Ganges river dolphin is associated with the deity Ganga.
Running evaluation for item: bb113f94-7723-47c6-8c34-59d883044514 (Input: {'text': 'What significant discovery did the LHCb collaboration report in 2015?'})
Generated Answer:  In 2015, the LHCb collaboration reported the discovery of pentaquark particles.
Running evaluation for item: 4d8ae54e-ceab-46d0-ad2c-6e8e223589a9 (Input: {'text': 'What is the MÄ\x81ori name for the red-crowned parakeet?'})
Generated Answer:  The Māori name for the red-crowned parakeet is kākāriki.
Running evaluation for item: 21e5a0d5-f619-4a73-868e-9955053b3e72 (Input: {'text': 'Who starred in the 1978 television film adaptation of Les MisÃ©rables?'})
Generated Answer:  Richard Jordan starred as Jean Valjean in the 1978 television film adaptation of Le

Σημαντικοί κανόνες:  
1. Μην προσθέσετε '''markdown ή οποιαδήποτε άλλα tags γύρω από τη μετάφραση.  
2. Βεβαιωθείτε ότι η μετάφραση δεν ακούγεται υπερβολικά κυριολεκτική.  
3. Μεταφράστε και τα σχόλια.  
4. Αυτό το αρχείο είναι γραμμένο σε μορφή Markdown - μην το αντιμετωπίσετε ως XML ή HTML.  
5. Μην μεταφράσετε:  
   - [!NOTE], [!WARNING], [!TIP], [!IMPORTANT], [!CAUTION]  
   - Ονόματα μεταβλητών, ονόματα συναρτήσεων, ονόματα κλάσεων  
   - Ενδιάμεσες θέσεις όπως @@INLINE_CODE_x@@ ή @@CODE_BLOCK_x@@  
   - URLs ή διαδρομές  
6. Διατηρήστε όλη την αρχική μορφοποίηση Markdown ανέπαφη.  
7. Επιστρέψτε ΜΟΝΟ το μεταφρασμένο περιεχόμενο χωρίς πρόσθετα tags ή markup.  

Μπορείτε να επαναλάβετε αυτή τη διαδικασία με διαφορετικές διαμορφώσεις agent όπως:  
- Μοντέλα (gpt-4o-mini, gpt-4.1, κ.λπ.)  
- Προτροπές  
- Εργαλεία (αναζήτηση έναντι χωρίς αναζήτηση)  
- Πολυπλοκότητα agent (πολλαπλοί agents έναντι ενός agent)  

Στη συνέχεια, συγκρίνετε τα δίπλα-δίπλα στο Langfuse. Σε αυτό το παράδειγμα, έτρεξα τον agent 3 φορές στις 25 ερωτήσεις του dataset. Για κάθε εκτέλεση, χρησιμοποίησα διαφορετικό μοντέλο Azure OpenAI. Μπορείτε να δείτε ότι ο αριθμός των σωστά απαντημένων ερωτήσεων βελτιώνεται όταν χρησιμοποιείται ένα μεγαλύτερο μοντέλο (όπως αναμενόταν). Το σκορ `correct_answer` δημιουργείται από έναν [LLM-as-a-Judge Evaluator](https://langfuse.com/docs/scores/model-based-evals) που έχει ρυθμιστεί να αξιολογεί την ορθότητα της ερώτησης βάσει της δείγματος απάντησης που δίνεται στο dataset.  

![Επισκόπηση εκτέλεσης dataset](https://langfuse.com/images/cookbook/example-autogen-evaluation/dataset_runs.png)  
![Σύγκριση εκτέλεσης dataset](https://langfuse.com/images/cookbook/example-autogen-evaluation/dataset-run-comparison.png)  



---

**Αποποίηση ευθύνης**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία αυτόματης μετάφρασης AI [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που καταβάλλουμε προσπάθειες για ακρίβεια, παρακαλούμε να έχετε υπόψη ότι οι αυτόματες μεταφράσεις ενδέχεται να περιέχουν λάθη ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα θα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρεξηγήσεις ή εσφαλμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
